Name: Mohini Khedekar
Date: April 13th 2025
Data Science Project 3
Link: https://colab.research.google.com/drive/1Z_fxY_cXVsYBr-YoOiDny0rcDw66xRFP?usp=sharing

Use the template below to train and assess NN-based classification models for the prediction of breast cancer subtypes, following the overall strategy and R template used for Project 1 while replacing logistic regression with NN models.

Step 1a: Copy project1 files to your Google Drive/data_science directory and use those files to train and assess NN classifiers

In [1]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')
!ls /content/drive/MyDrive/data_science/

# Copy the file to be used for the project from your data_science directory
cancer_dataset_name = "TCGA_breast_cancer_LumA_vs_Basal_PAM50.tsv"
!cp /content/drive/MyDrive/data_science/$cancer_dataset_name .
!head -10 $cancer_dataset_name

# Read the TSV data into a pandas DataFrame
data = pd.read_csv(cancer_dataset_name, sep='\t')
print(data.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
TCGA_breast_cancer_ERpositive_vs_ERnegative_PAM50.tsv
TCGA_breast_cancer_ERstatus_allGenes.txt
TCGA_breast_cancer_LumA_vs_Basal_PAM50.tsv
sample_id	TCGA-A7-A13E-01A-11R-A12P-07	TCGA-A8-A08H-01A-21R-A00Z-07	TCGA-A8-A07U-01A-11R-A034-07	TCGA-BH-A0E0-01A-11R-A056-07	TCGA-A2-A04T-01A-21R-A034-07	TCGA-BH-A18K-01A-11R-A12D-07	TCGA-AQ-A04J-01A-02R-A034-07	TCGA-A1-A0SK-01A-12R-A084-07	TCGA-A7-A0DA-01A-31R-A115-07	TCGA-AR-A1AI-01A-11R-A12P-07	TCGA-E2-A14X-01A-11R-A115-07	TCGA-A2-A0D0-01A-11R-A00Z-07	TCGA-AN-A0FX-01A-11R-A034-07	TCGA-E2-A14Y-01A-21R-A12D-07	TCGA-AR-A0TU-01A-31R-A109-07	TCGA-D8-A143-01A-11R-A115-07	TCGA-AR-A0U4-01A-11R-A109-07	TCGA-AR-A0U0-01A-11R-A109-07	TCGA-A2-A0SX-01A-12R-A084-07	TCGA-B6-A0I6-01A-11R-A034-07	TCGA-E2-A159-01A-11R-A115-07	TCGA-BH-A0B9-01A-11R-A056-07	TCGA-E2-A150-01A-11R-A12D-07	TCGA-A8-A07R-01A-21R-A034-07	TCGA-AO-A0JL-01A-11R-A056-0

Step 2: Perform the necessary parsing to get what you need in order

In [2]:
# Extract features (gene expression values) and target classes (subtype)
# Samples are all columns except the first one (which contains gene names)
# Features are all rows except the first two (that contain sample ID and class)
X = data.iloc[1:, 1:].T.values  # Features (gene expression values)
y = data.iloc[0, 1:].values  # Target classes
print(X)
print(y)
# Convert target labels to numerical values (0 for Basal-like, 1 for Luminal A)
y = [0 if label == 'Basal-like' else 1 for label in y]


[['3.59727' '8.38208' '11.0157' ... '3.37778' '9.02075' '4.2101']
 ['7.2052' '8.53294' '10.3315' ... '9.81946' '7.53126' '5.43699']
 ['5.12902' '9.01439' '10.5027' ... '7.64791' '10.4528' '5.38517']
 ...
 ['10.5333' '6.69209' '10.5999' ... '13.2352' '6.5445' '7.72875']
 ['11.9222' '8.26247' '10.6069' ... '13.8731' '7.21966' '9.78624']
 ['13.7303' '7.38246' '10.661' ... '13.8056' '7.82931' '7.21018']]
['Basal-like' 'Basal-like' 'Basal-like' 'Basal-like' 'Basal-like'
 'Basal-like' 'Basal-like' 'Basal-like' 'Basal-like' 'Basal-like'
 'Basal-like' 'Basal-like' 'Basal-like' 'Basal-like' 'Basal-like'
 'Basal-like' 'Basal-like' 'Basal-like' 'Basal-like' 'Basal-like'
 'Basal-like' 'Basal-like' 'Basal-like' 'Basal-like' 'Basal-like'
 'Basal-like' 'Basal-like' 'Basal-like' 'Basal-like' 'Basal-like'
 'Basal-like' 'Basal-like' 'Basal-like' 'Basal-like' 'Basal-like'
 'Basal-like' 'Basal-like' 'Basal-like' 'Basal-like' 'Basal-like'
 'Basal-like' 'Basal-like' 'Basal-like' 'Basal-like' 'Basal-like'
 '

Step 3: Use logistic regression to learn and asses classification models within cross-validation

In [3]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score
import numpy as np
import warnings

# Suppress warnings from logistic regression
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn.linear_model")


# Test logistic regression with a simple split into training and test sets
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train a logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Test logistic regression using cross-validation
# Perform 5-fold cross-validation

# Create a StratifiedKFold object with 5 splits
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform stratified cross-validation
model = LogisticRegression()
scores = cross_val_score(model, X, y, cv=skf, scoring='accuracy')

# Compute mean accuracy and standard deviation
mean_accuracy = np.mean(scores)
std_accuracy = np.std(scores)

print(f"Mean Accuracy: {mean_accuracy}")
print(f"Standard Deviation: {std_accuracy}")


Accuracy: 0.9545454545454546
Mean Accuracy: 0.9663403263403264
Standard Deviation: 0.022574214151553078


Step 4: Use a simple feed-forward neural network to train and assess classification models within cross-validation

In [4]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.neural_network import MLPClassifier
import numpy as np
import warnings

# Suppress warnings
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn.neural_network")


# Create a StratifiedKFold object with 5 splits
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Create and train a neural network model with one hidden layer (10 nodes)
model = MLPClassifier(hidden_layer_sizes=(10,), activation='relu', solver='adam', random_state=42)

# Perform stratified cross-validation
scores = cross_val_score(model, X, y, cv=skf, scoring='accuracy')

# Compute mean accuracy and standard deviation
mean_accuracy = np.mean(scores)
std_accuracy = np.std(scores)

print(f"Mean Accuracy: {mean_accuracy}")
print(f"Standard Deviation: {std_accuracy}")


Mean Accuracy: 0.9785547785547786
Standard Deviation: 0.018415803584402812


As can be seen through the mean accuracy values, neural networks perform better than simple logistic regression based classification to categorize luminal A vs basal cancer subtypes. As can be seen through a lower standard deviation value for the neural network model, it is more stable across the folds.

Step 5: Play with meta-parameters in the model, including different activation functions, more hidden layers, more or less nodes in hidden layers to change the complexity of the model and its impact on the results.

In [5]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.neural_network import MLPClassifier
import numpy as np
import warnings

# Suppress warnings
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn.neural_network")

# Cross-validation setup
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


#Code written with the help of ChatGPT
# Different model configs to try
configs = [
    {"hidden_layer_sizes": (10,), "activation": "relu"},

    #increasing the number of nodes
    {"hidden_layer_sizes": (50,), "activation": "relu"},
    {"hidden_layer_sizes": (100,), "activation": "relu"},

    #adding more hidden layers with varying number of nodes and different activation functions
    {"hidden_layer_sizes": (50, 25), "activation": "relu"},
    {"hidden_layer_sizes": (30, 30, 30), "activation": "tanh"},
    {"hidden_layer_sizes": (64, 32), "activation": "logistic"},
    {"hidden_layer_sizes": (100, 50, 10), "activation": "relu"},
]

# Run cross-validation for each config
for idx, config in enumerate(configs):
    model = MLPClassifier(
        hidden_layer_sizes=config["hidden_layer_sizes"],
        activation=config["activation"],
        solver='adam',
        max_iter=500,
        random_state=42
    )

    scores = cross_val_score(model, X, y, cv=skf, scoring='accuracy')
    mean_accuracy = np.mean(scores)
    std_accuracy = np.std(scores)

    print(f"\nModel {idx + 1}: hidden_layers={config['hidden_layer_sizes']}, activation={config['activation']}")
    print(f"Mean Accuracy: {mean_accuracy:.4f}")
    print(f"Standard Deviation: {std_accuracy:.4f}")



Model 1: hidden_layers=(10,), activation=relu
Mean Accuracy: 0.9786
Standard Deviation: 0.0184

Model 2: hidden_layers=(50,), activation=relu
Mean Accuracy: 0.9725
Standard Deviation: 0.0150

Model 3: hidden_layers=(100,), activation=relu
Mean Accuracy: 0.9786
Standard Deviation: 0.0184

Model 4: hidden_layers=(50, 25), activation=relu
Mean Accuracy: 0.9755
Standard Deviation: 0.0208

Model 5: hidden_layers=(30, 30, 30), activation=tanh
Mean Accuracy: 0.9724
Standard Deviation: 0.0225

Model 6: hidden_layers=(64, 32), activation=logistic
Mean Accuracy: 0.9786
Standard Deviation: 0.0184

Model 7: hidden_layers=(100, 50, 10), activation=relu
Mean Accuracy: 0.9724
Standard Deviation: 0.0180


The performance of the model remains robust to changes in the parameters like number of hidden layers, the number of nodes in each of the hidden layers and also the type of activation function used. This suggests that adding complexity to this model does not necessarily improve generalization since this dataset is relatively easy to classify. The best model was the one with one hidden layer and 100 nodes as well as the one with two hidden layers with 64 and 32 nodes respectively. This is the architecture I am going to use for the next part of the project considering the fact that the ER status dataset is larger and more complicated.

Step 6: Note that Basal-like vs. Luminal A subtype classification is an easy task as these two classes represent two most distinct molecular subtypes of breast cancer. Note also that the problem is further simplified by using an oracle provided magic set of 50 features (genes) that separate these subtypes very well. Following Project 1, expand the code to perform both logistic regression and neural network training and assessment within cross-validation on the more challenging task of predicting ER status using all genes with t-test based feature selection to limit the set of genes to 1000, 500, 100, and compare the results. Make sure that your feature selection is done on the training subsets only!

In [18]:
# Uploading the file manually since colab was not connecting to google drive
from google.colab import files
uploaded = files.upload()

Saving TCGA_breast_cancer_ERstatus_allGenes.txt to TCGA_breast_cancer_ERstatus_allGenes.txt


In [6]:

import pandas as pd

#Load the dataset
data = pd.read_csv("TCGA_breast_cancer_ERstatus_allGenes.txt", sep="\t")
print(data.head())

<ipython-input-6-8c5f881deb1b>:4: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263

               id TCGA-A7-A13E-01A-11R-A12P-07 TCGA-A8-A08H-01A-21R-A00Z-07  \
0  class-ERstatus                     Positive                     Positive   
1            A1BG                  6.862111631                  7.194049613   
2             A2M                  12.27289156                  15.28033619   
3            NAT1                  3.597268951                  7.205198947   
4            NAT2                  0.512985335                   2.56078831   

  TCGA-A8-A07U-01A-11R-A034-07 TCGA-BH-A0E0-01A-11R-A056-07  \
0                     Negative                     Negative   
1                   4.40083163                  5.849819161   
2                  13.94199511                  12.83033137   
3                  5.129023308                  4.828951401   
4                  0.485323774                  3.848647976   

  TCGA-A2-A04T-01A-21R-A034-07 TCGA-BH-A18K-01A-11R-A12D-07  \
0                     Negative                     Positive   
1                  6

In [7]:
# Extracting the target labels (ER status) from the first row (excluding the 'id' column)
y = data.iloc[0, 1:].values  # ER status for each sample

# Extracting gene expression values from all rows except the first (gene names)
X = data.iloc[1:, 1:].T.values  # Gene expression values (features)
print(X)
print(y)

# Convert ER status labels to binary: 1 for Positive, 0 for Negative
y = [1 if label == 'Positive' else 0 for label in y]


[['6.862111631' '12.27289156' '3.597268951' ... 2.14280551 4.711208542
  4.824258697]
 ['7.194049613' '15.28033619' '7.205198947' ... 0.919149363 5.008267255
  4.127872609]
 ['4.40083163' '13.94199511' '5.129023308' ... 1.378345149 4.666500974
  3.743913448]
 ...
 ['7.928911528' '13.84074778' '10.53327862' ... 3.498084884 6.473817752
  4.931815408]
 ['4.42947537' '14.39031168' '11.92216573' ... 2.648166289 5.658948355
  4.39376626]
 ['7.039138394' '12.39786509' '13.73026968' ... 3.055386566 4.918977694
  5.620949971]]
['Positive' 'Positive' 'Negative' 'Negative' 'Negative' 'Positive'
 'Negative' 'Negative' 'Negative' 'Negative' 'Negative' 'Negative'
 'Negative' 'Positive' 'Negative' 'Negative' 'Negative' 'Negative'
 'Negative' 'Negative' 'Negative' 'Negative' 'Negative' 'Negative'
 'Negative' 'Negative' 'Negative' 'Negative' 'Negative' 'Negative'
 'Negative' 'Positive' 'Negative' 'Positive' 'Negative' 'Negative'
 'Negative' 'Negative' 'Negative' 'Negative' 'Negative' 'Negative'
 'Negat

LOGISTIC REGRESSION TO CLASSIFY SAMPLES AS ER+/ER-

In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from scipy import stats
import warnings

# Suppress warnings from logistic regression (e.g., convergence warnings)
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn.linear_model")

# Ensure X and y are numpy arrays
X = np.array(X)
y = np.array(y)

# Define different numbers of top genes to test
top_gene_counts = [1000, 500, 100]

# Iterate over each top_gene threshold
for top_genes in top_gene_counts:
    print(f"\nEvaluating for Top {top_genes} genes")

    # Store accuracy scores for this gene count
    cv_accuracies = []

    # Test logistic regression using cross-validation
    # Perform 5-fold cross-validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    for train_index, test_index in skf.split(X, y):
        # Split data
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Perform t-test for each gene
        p_values = []
        for gene_idx in range(X_train.shape[1]):
            try:
                _, p = stats.ttest_ind(X_train[y_train == 0, gene_idx], X_train[y_train == 1, gene_idx], equal_var=False)
            except:
                p = 1.0  # Assign high p-value if test fails
            p_values.append(p)

        # Select top genes
        top_gene_indices = np.argsort(p_values)[:top_genes]
        X_train_selected = X_train[:, top_gene_indices]
        X_test_selected = X_test[:, top_gene_indices]

        # Train and evaluate model
        model = LogisticRegression(max_iter=1000)
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        acc = accuracy_score(y_test, y_pred)
        cv_accuracies.append(acc)


    # Compute mean accuracy and standard deviation
    mean_acc = np.mean(cv_accuracies)
    std_acc = np.std(cv_accuracies)
    print(f"Mean Accuracy: {mean_acc:.4f}")
    print(f"Standard Deviation: {std_acc:.4f}")



Evaluating for Top 1000 genes
Mean Accuracy: 0.9505
Standard Deviation: 0.0149

Evaluating for Top 500 genes
Mean Accuracy: 0.9350
Standard Deviation: 0.0117

Evaluating for Top 100 genes
Mean Accuracy: 0.9318
Standard Deviation: 0.0235


NEURAL NETWORK MODEL TO CLASSIFY SAMPLES AS ER+/ER-

In [9]:
# Define different numbers of top genes to test
top_gene_counts = [1000, 500, 100]

# Iterate over each top_gene threshold
for top_genes in top_gene_counts:
    print(f"\nEvaluating for Top {top_genes} genes")

    # Store accuracy scores for this gene count
    cv_accuracies = []

    # Test logistic regression using cross-validation
    # Perform 5-fold cross-validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    for train_index, test_index in skf.split(X, y):
        # Split data
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Perform t-test for each gene
        p_values = []
        for gene_idx in range(X_train.shape[1]):
            try:
                _, p = stats.ttest_ind(X_train[y_train == 0, gene_idx], X_train[y_train == 1, gene_idx], equal_var=False)
            except:
                p = 1.0  # Assign high p-value if test fails
            p_values.append(p)

        # Select top genes
        top_gene_indices = np.argsort(p_values)[:top_genes]
        X_train_selected = X_train[:, top_gene_indices]
        X_test_selected = X_test[:, top_gene_indices]

        # Define the neural network model
        model = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', random_state=42)

        # Train the model
        model.fit(X_train_selected, y_train)

        # Predict and evaluate
        y_pred = model.predict(X_test_selected)
        acc = accuracy_score(y_test, y_pred)
        cv_accuracies.append(acc)

    # Compute mean and std of accuracies
    mean_acc = np.mean(cv_accuracies)
    std_acc = np.std(cv_accuracies)

    print(f"Mean Accuracy: {mean_acc:.4f}")
    print(f"Standard Deviation: {std_acc:.4f}")



Evaluating for Top 1000 genes
Mean Accuracy: 0.9443
Standard Deviation: 0.0186

Evaluating for Top 500 genes
Mean Accuracy: 0.9381
Standard Deviation: 0.0097

Evaluating for Top 100 genes
Mean Accuracy: 0.9320
Standard Deviation: 0.0154


The logistic regression and the neural network model perform pretty similarly as can be depicted by the mean accuracy values and their standard deviation. Both the logisitc regression model and the neural network model perform the best with top features = 1000. This is because with fewer features, some informative genes are excluded. The best model here is the logistic regression model with top selected features = 1000. The neural networks display a slightly more consistent performance and also perform slightly better than logistic regression when it comes to classifying samples as ER+/ER- with only 100 features. I also believe a neural network model with a more complex architecture will outperform the logistic regression model for the same number of features. Additionally, we can further analyze whether the models are underfitting/overfitting by analzying the training vs validation accuracy and printing learning curves for both training and validation.